In [1]:
from jaxformer.hf.sample import * 

/home/PracticalWork2021/CodeGen/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class args:
    def __init__(self):
        self.fp16 = True
        self.model = 'codegen-16B-mono'
        self.device = 'cuda:0'
        self.rng_seed = 42
        self.rng_deterministic = True
        self.pad = 50256
        self.context = 'def helloworld():'
        self.batch_size = 1
        self.max_length = 128
        self.t = 0.2
        self.p = 0.95

args = args()

In [3]:
print(args.model)

codegen-16B-mono


In [4]:
models_nl = []
models_pl = ['codegen-350M-mono', 'codegen-2B-mono', 'codegen-6B-mono', 'codegen-16B-mono']
models = models_nl + models_pl

In [12]:
from datasets import load_dataset
human_eval_dataset = load_dataset('openai_humaneval')['test']


Reusing dataset openai_humaneval (/root/.cache/huggingface/datasets/openai_humaneval/openai_humaneval/1.0.0/2955cebd73602e828fa8c0a424c594e5fab4ec863b316ca98f3d8fdb6a626e75)
100%|██████████| 1/1 [00:00<00:00, 827.77it/s]


In [13]:
name = "asdiv_prefix"

file = "../data/priming_texts/asdiv/"+ name +".txt"


with open(file, "r") as f:
    prompt = f.read()

args.context = human_eval_dataset[67]['prompt']
print(args.context)



def fruit_distribution(s,n):
    """
    In this task, you will be given a string that represents a number of apples and oranges 
    that are distributed in a basket of fruit this basket contains 
    apples, oranges, and mango fruits. Given the string that represents the total number of 
    the oranges and apples and an integer that represent the total number of the fruits 
    in the basket return the number of the mango fruits in the basket.
    for examble:
    fruit_distribution("5 apples and 6 oranges", 19) ->19 - 5 - 6 = 8
    fruit_distribution("0 apples and 1 oranges",3) -> 3 - 0 - 1 = 2
    fruit_distribution("2 apples and 3 oranges", 100) -> 100 - 2 - 3 = 95
    fruit_distribution("100 apples and 1 oranges",120) -> 120 - 100 - 1 = 19
    """



In [6]:
set_env()
set_seed(args.rng_seed, deterministic=args.rng_deterministic)

device = torch.device(args.device)
ckpt = f'./checkpoints/{args.model}'

# (3) load

with print_time('loading parameters'):
    model = create_model(ckpt=ckpt, fp16=args.fp16).to(device)


with print_time('loading tokenizer'):
    if args.model in models_pl:
        tokenizer = create_custom_gpt2_tokenizer()
    else:
        tokenizer = create_tokenizer()
    tokenizer.padding_side = 'left'
    tokenizer.pad_token = args.pad

loading parameters
loading parameters took 464.87s
loading tokenizer
loading tokenizer took 4.07s


In [14]:
with print_time('sampling'):
        completion = sample(device=device, model=model, tokenizer=tokenizer, context=args.context, pad_token_id=args.pad, num_return_sequences=args.batch_size, temp=args.t, top_p=args.p, max_length_sample=args.max_length)[0]
        truncation = truncate(completion)

        print('=' * 100)
        print(completion)
        #print('=' * 100)
        #print(args.context+truncation)
        #print('=' * 100)

sampling
    #
sampling took 0.24s
